## Import packages

In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import torch
sys.path.append('./')  # uncomment for local import
import tangram as tg
import copy
import anndata

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

tg.__version__

'1.0.3'

## Preprocess ish and d

## Create/Import anndata

### Spacial transcriptomics data

In [ ]:
adata = sc.read_h5ad("/beegfs/scratch/bruening_scratch/lsteuernagel/data/whole_brain_atlas/Macosko/spatial/macosko_hypo_pucks.h5ad")
adata = adata[(adata.obs['nCount_Spatial'] > 300) & (adata.obs['nCount_Spatial'] < 5000)]
adata.__dict__['_raw'].__dict__['_var'] = adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
del adata.raw 

### Single cell data

In [23]:
hm_sc = sc.read_h5ad("/beegfs/scratch/bruening_scratch/lsteuernagel/data/hypoMap_publication/hypoMap.h5ad")

AnnData object with n_obs × n_vars = 384925 × 57362
    obs: 'Cell_ID', 'Dataset', 'SRA_ID', 'Sample_ID', 'GEO_ID', 'Run10x', 'Technology', 'Strain', 'Diet', 'Pooled', 'Age', 'Author_Region', 'inferred_sex', 'nCount_RNA', 'nFeature_RNA', 'percent_mt', 'Author_Exclude', 'Author_Class', 'Author_CellType', 'percent_exclude_features', 'S.Score', 'G2M.Score', 'Phase', 'Batch_ID', 'Author_Condition', 'Sex', 'Author_Batch', 'Author_Class_Curated', 'C2', 'C7', 'C25', 'C66', 'C185', 'C286', 'C465', 'C2_named', 'C7_named', 'C25_named', 'C66_named', 'C185_named', 'C286_named', 'C465_named', 'Region_predicted', 'Region_summarized'
    var: 'features'
    uns: 'neighbors'
    obsm: 'X_scvi', 'X_umap_scvi'
    obsp: 'distances'

### Gene markers

#### DEG

In [ ]:
sc.tl.rank_genes_groups(hm_sc, groupby="C185_named", use_raw=False, method='wilcoxon')
markers_df = pd.DataFrame(hm_sc.uns["rank_genes_groups"]["names"]).iloc[0:200, :]
markers = list(np.unique(markers_df.melt().value.values))
len(markers)

genes_to_exclude = pd.read_csv('/beegfs/scratch/bruening_scratch/lsteuernagel/projects/analysis_projects/volumetric_analysis/genes_to_exclude.csv',
                               header = None)
markers = pd.DataFrame(markers)[~pd.DataFrame(markers).iloc[:,0].isin(genes_to_exclude.iloc[:,0])]
len(markers)

markers.to_csv("/beegfs/home/pmatyskova/project/hypomap_diffwilcoxon_top200.csv") #top50, 100, 200

In [24]:
markers = pd.read_csv('/beegfs/home/pmatyskova/project/hypomap_diffwilcoxon_top200.csv') #top50, 100, 200
markers = np.array(markers['0'])

#### MRx3 approach

In [10]:
df_genes = np.load("/beegfs/home/pmatyskova/project/mrmrstep_geneset_miss_hm185cor_minres_mrx31500.npy", 
                   allow_pickle=True)
df_genes = pd.DataFrame(df_genes)

genes_to_exclude = pd.read_csv('/beegfs/scratch/bruening_scratch/lsteuernagel/projects/analysis_projects/volumetric_analysis/genes_to_exclude.csv',
                               header = None)
df_genes = df_genes[~df_genes.iloc[:,0].isin(genes_to_exclude.iloc[:,0])] #exclude the genes to exclude from hypomap output

markers = np.reshape(df_genes.values, (-1, ))
markers= list(markers)
markers = markers[0:1300]
len(markers)

['Sparc',
 'Cldn1',
 'Cdh11',
 'Prnp',
 'Cldn11',
 'Pmch',
 'Npy',
 'Rorb',
 'Foxp2',
 'Slc17a6',
 'Gal',
 'Sema6d',
 'Zic1',
 'Ptn',
 'Nxph1',
 'Pomc',
 'Avp',
 'Slc32a1',
 'Mef2c',
 'Serpine2',
 'Kcnip4',
 'Gpc3',
 'Tcf4',
 'Ptprd',
 'Otp',
 'Cartpt',
 'Rgs10',
 'Cd9',
 'Stmn4',
 'Tac1',
 'Sncg',
 'Tac2',
 'Opcml',
 'Ntrk2',
 'Isl1',
 'Arpp21',
 'Bsg',
 'Igfbp5',
 'Cacna2d1',
 'Ptprz1',
 'Lingo2',
 'Rab3b',
 'Ntm',
 'Dbi',
 'Pex5l',
 'Calb1',
 'Plxdc2',
 'Tenm2',
 'Cck',
 'Synpr',
 'Id4',
 'Fxyd6',
 'Meis2',
 'Nrp1',
 'Tnr',
 'Gm13889',
 'C1qc',
 'Cryab',
 'S100a10',
 'Aldoc',
 'Brinp3',
 'Nts',
 'Slc6a1',
 'Grid2',
 'Cnp',
 'Ntng1',
 'Utrn',
 'Erbb4',
 'Gfra1',
 'S100a6',
 'Ctss',
 'Gatm',
 'Lrp1b',
 'Slc1a2',
 'Ptprk',
 'Grm5',
 'Unc13c',
 'Dlx1',
 'Sox2ot',
 'Ly6e',
 'Elavl2',
 'Pcdh9',
 'Parm1',
 'Timp3',
 'Gpm6b',
 'Slit2',
 'Alcam',
 'Bcan',
 'Lhfpl3',
 'Tmeff2',
 'Tcf7l2',
 'Msi2',
 'Adarb2',
 'S100a16',
 'Higd1a',
 'Cadm2',
 'Asb4',
 'Gas6',
 'Vim',
 'Kcnd2',
 'Rprm',
 'Gpr37

#### The rest of the gene marker pipeline

In [30]:
tg.pp_adatas(hm_sc, adata, genes=markers)
#tg.pp_adatas(hm_sc, ish_sp)

INFO:root:3310 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:11709 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [31]:
assert hm_sc.uns['training_genes'] == adata.uns['training_genes']

## Train the model

In [ ]:
ad_map = tg.map_cells_to_space(adata_sc=hm_sc, adata_sp=adata, mode='clusters', #mode='constrained'
                               cluster_label = 'C185_named') #, lambda_r = 0.1)
                               #lambda_f_reg = 1, target_count = 2031) #C286 ir C185

## Save the results

In [10]:
np.save(file = "/beegfs/home/pmatyskova/project/spatial_tangram_mrx31300_alls", 
        arr = np.transpose(ad_map.X)) #not transpose in mode='constrained'
ad_map.obs.to_csv("/beegfs/home/pmatyskova/project/spatialobs_tangram_mrx31300_alls.csv")
ad_map.var.to_csv("/beegfs/home/pmatyskova/project/spatialvar_tangram_mrx31300_alls.csv")
#ad_map.write_csvs('/beegfs/home/pmatyskova/project/d_tangram_hm185cor_minres_full') #for mode='constrained'

### Open saved results (for model constrained that was trained on servers)

In [5]:
ad_map = anndata.AnnData(
    np.transpose(np.load("/beegfs/home/pmatyskova/project/spatial_tangram_mrx31300_alls.npy")),
    pd.read_csv('/beegfs/home/pmatyskova/project/spatialobs_tangram_mrx31300_alls.csv'),
    pd.read_csv('/beegfs/home/pmatyskova/project/spatialvar_tangram_mrx31300_alls.csv'),
    )

In [7]:
ad_map.obs.index=ad_map.obs['Unnamed: 0']
ad_map.var.index=ad_map.var['Unnamed: 0']
ad_map

AnnData object with n_obs × n_vars = 185 × 81829
    obs: 'Unnamed: 0', 'C185_named', 'cluster_density'
    var: 'Unnamed: 0', 'nCount_Spatial', 'nFeature_Spatial', 'PuckID', 'Raw_Slideseq_X', 'Raw_Slideseq_Y', 'Puck_Depth', 'nCount_Spatial.1', 'nFeature_Spatial.1', 'CCF_X', 'CCF_Y', 'CCF_Z', 'TopStruct', 'DeepCCF', 'CCF_acronym', 'CCF_ID', 'CCF_Name', 'CCF_LeftRight', 'IsOutsideCCF', 'Correponding_DissectateSegName', 'Correponding_DissectateSegID', 'NisslTiffX', 'NisslTiffY', 'BeadBarcode', '.id', 'orig.ident', 'Bead_ID', 'uniform_density', 'rna_count_based_density'

## Quantitative model evaluation

### Fit back

In [11]:
ad_ge = tg.project_genes(
                  ad_map, 
                  hm_sc, #hm_sct for mode='constrained'
                  cluster_label='C185_named') 

In [12]:
df_all_genes = tg.compare_spatial_geneexp(ad_ge, adata, hm_sc) #hm_sct

In [13]:
df_all_genes.to_csv("/beegfs/home/pmatyskova/project/spatialcorr_tangram_mrx31300_alls.csv") 

### Ground truth evaluation

In [8]:
def annot_function(d_matrix, ref): #d_matrix in mode='cluster', d in mode='constrained'
    #annotate predictions (cell type & voxel locations + ABA annotations)
    d = pd.DataFrame(np.transpose(d_matrix.X)) #from anndata to pandas, comment in mode='constrained'
    d.columns = d_matrix.obs['C185_named'] #comment in mode='constrained
    d = d.loc[:,d.columns.isin(ref.loc[:,'cluster'])] #only keep cell types for which we have reference
    d['name'] = np.array(ad_map.var['CCF_Name']) #index column for merging
    
    return(d)

In [9]:
def eval_function(d_matrix, ref):    
    #evaluation (comparson with the ground truth)
    d_ann = annot_function(d_matrix, ref)
    
    model_eval = []
    for i in range(ref.shape[0]):
        #filter region that is predicted in the ground truth for each cell type
        #to include not only exact region name but also its children - not just "Medial preoptic nucleus"
        #but also Medial preoptic nucleus, central/lateral/medial part:
        filt = [] 
        for j in range(d_matrix.shape[1]): #1 in mode='cluster', 0 in mode='constrained'
            filt_i = ref['Region_ground_truth'][i] in d_ann['name'][j]
            filt.append(filt_i)
        d_filt = d_ann[filt]
    
        #calculations:
        score_i = sum(d_filt[ref['cluster'][i]])
        model_eval.append(score_i)
    
    copy_ref = copy.copy(ref)
    copy_ref['model_eval'] = model_eval
    return(copy_ref)

In [10]:
gt_hm185 = pd.read_csv('/beegfs/scratch/bruening_scratch/lsteuernagel/projects/analysis_projects/volumetric_analysis/hypoMap_region_annotation_withSpatial_C185.txt', sep = "\t")

gt_hm185 = gt_hm185.iloc[:,0:2]

In [11]:
gt_hm185 = gt_hm185.drop_duplicates(subset = ['cluster'], keep='first')
gt_hm185.index = np.arange(0,len(gt_hm185))

In [12]:
final_eval = eval_function(ad_map, gt_hm185)
final_eval

,cluster,Region_ground_truth,model_eval
0,C185-65: Unassigned.Mixed.GABA-2,Medial preoptic nucleus,0.073465
1,C185-71: Vip.Vipr2.GABA-2,Suprachiasmatic nucleus,0.014247
2,C185-72: Fam122b.Vipr2.GABA-2,Suprachiasmatic nucleus,0.004654
3,C185-73: Cck.Vipr2.GABA-2,Suprachiasmatic nucleus,0.006504
4,C185-11: Cbln2.Trh.GLU-2,Paraventricular hypothalamic nucleus,0.058125
...,...,...,...
63,C185-134: Frzb.Tanycytes,Arcuate hypothalamic nucleus,0.051540
64,C185-51: Tac2.GLU-5,Arcuate hypothalamic nucleus,0.037020
65,C185-61: Prkch.GLU-8,Lateral mammillary nucleus,0.302605
66,C185-64: Meis2.Mixed.GABA-2,Zona incerta,0.319921


## Permutation test

### Permutation test on voxel randomised prediction matrix

In [13]:
def voxelperm_annot_function(d_matrix, ref): #d_matrix in eval mode='cluster', d in mode='constrained'
    #annotate voxel permuted predictions (cell type & randomised voxel location + ABA annotations)
    d = pd.DataFrame(np.transpose(d_matrix.X)) #from anndata to pandas, comment in mode='constrained'
    d.columns = d_matrix.obs['C185_named'] #C286 or C185, comment in mode='constrained'
    d = d.loc[:,d.columns.isin(ref.loc[:,'cluster'])] #only keep cell types for which we have reference
    
    d_annot = d.sample(frac=1, axis=0) #suffle row order
    d_annot.index = d.index
    
    d_annot['name'] = np.array(ad_map.var['CCF_Name'])
    
    model_eval = []
    for i in range(ref.shape[0]):
        #filter region that is predicted in the ground truth for each cell type
        #to include not only exact region name but also its children - not just "Medial preoptic nucleus"
        #but also Medial preoptic nucleus, central/lateral/medial part:
        filt = [] 
        for j in range(d_matrix.shape[1]): #1 & d_matrix in mode='cluster', 0 & d in mode='constrained'
            filt_i = ref['Region_ground_truth'][i] in d_annot['name'][j]
            filt.append(filt_i)
        d_filt = d_annot[filt]
    
        #calculations:
        score_i = sum(d_filt[ref['cluster'][i]])
        model_eval.append(score_i)
    
    copy2_ref = copy.copy(ref)
    copy2_ref['randmodel_eval'] = model_eval
    return(copy2_ref)

In [14]:
voxpermutation_iters = 200
voxpermut_evals = pd.DataFrame(columns = list(map('x{}'.format, range(1, voxpermutation_iters+1))))
for i in range(voxpermutation_iters):
    print(i)
    dvoxperm_evaluation = voxelperm_annot_function(ad_map, gt_hm185) #gt_hm286 or gt_hm286n or gt_hm185
    voxpermut_evals.iloc[:,i] = dvoxperm_evaluation['randmodel_eval']

voxpermut_eval_tot = voxpermut_evals.mean(axis=1)
final_eval['voxpermut_eval'] = voxpermut_eval_tot
final_eval

0


KeyboardInterrupt: 

In [ ]:
final_eval.to_csv("/beegfs/home/pmatyskova/project/feval_spatial_tangram_mrx31300_alls.csv") 